In [3]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [4]:
# default_exp layers

In [59]:
#export
from fastai.imports import *
from fastai.torch_imports import *
from fastai.torch_core import *
from fastai.layers import *

In [6]:
#hide
from nbdev.showdoc import *

# Layers

> Some layers which tops up the ones in fastai

## Basic manipulations and resizing

One can easily create a beautiful layer with minimum boilerplate using fastai utilities. We will show a simple example here. For details and extensive illustrations please refer to [decorated fastai layers](https://docs.fast.ai/layers.html#Basic-manipulations-and-resize).

An easy way to create a pytorch layer for a simple `func`

In [57]:
def _add2(x): return x+2
tst = Lambda(_add2)
x = torch.randn(10,20)
test_eq(tst(x), x+2)
tst2 = pickle.loads(pickle.dumps(tst))
test_eq(tst2(x), x+2)

## BatchNorm layers

In [6]:
# export
class LinBnDrop(nn.Sequential):
    "Module grouping `BatchNorm1d`, `Dropout` and `Linear` layers"
    def __init__(self, n_in, n_out=None, bn=True, ln=True, p=0., act=None, lin_first=False):
        layers = [BatchNorm(n_out if ln and lin_first else n_in, ndim=1)] if bn else []
        if p != 0: layers.append(nn.Dropout(p))
        lin = [nn.Linear(n_in, n_out, bias=not bn)] if ln else []
        if ln and act is not None: lin.append(act)
        layers = lin+layers if lin_first else layers+lin
        super().__init__(*layers)

The `BatchNorm` or the `Linear` layer is skipped if `bn=False` or `ln=False`, as is the dropout if `p=0`. Optionally, you can add an activation for after the linear layer with act.

In [7]:
tst = LinBnDrop(10, 20)
list(tst.children())

[BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Linear(in_features=10, out_features=20, bias=False)]

In [8]:
tst = LinBnDrop(10, 20, ln=False, p=0.02)
tst

LinBnDrop(
  (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Dropout(p=0.02, inplace=False)
)

The `LinBnDrop` layer ia not going to add an activation if `ln` is `False`:

In [9]:
tst = LinBnDrop(10, 20, ln=False, p=0.02, act=nn.ReLU(inplace=True))
tst

LinBnDrop(
  (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Dropout(p=0.02, inplace=False)
)

## Attention Layers

In [85]:
# export
class XMLAttention(Module):
    "Compute label specific attention weights for each token in a sequence"
    def __init__(self, n_lbs, emb_sz):
         self.lbs_emb = nn.Embedding(n_lbs, emb_sz)
         self.lbs_emb.weight.data.normal_(0, 0.01)   

    def forward(self, x):
        attn_wgts = F.softmax(x @ self.lbs_emb.weight.transpose(0,1), dim=1)
        return attn_wgts.transpose(1,2) @ x
    

In [91]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_text.models.core.ipynb.
Converted 03_text.learner.ipynb.
Converted 04_metrics.ipynb.
Converted index.ipynb.
